# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import re
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [3]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\BOluga\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\BOluga\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
# load data from database

def load_data():
    engine = create_engine('sqlite:///DisasterResponse.db')
    sql = 'SELECT * from messages_and_categories;'
    df = pd.read_sql(sql, engine)
    
    X = df['message']
    y = df[['related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report']]

    return X, y

In [5]:
X, y = load_data()
X.sample(5)

18414    With many power pylons toppling over, the enti...
4766     Carrefour Feuilles needs food, drinking water ...
1446     If a person needs to go to St Domingo and does...
17448    Volunteers are also collecting and burying dea...
17620    It also says the potential for monopolisation ...
Name: message, dtype: object

In [6]:
y.sample(5)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
3182,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1546,1,1,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,1
21824,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21196,1,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1438,1,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0


### 2. Write a tokenization function to process your text data

In [7]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = [lemmatizer.lemmatize(token).lower().strip() for token in tokens]

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('knn_clf', MultiOutputClassifier(KNeighborsClassifier()))
])

knn_est_params = {
    #'knn_clf__estimator__algorithm': ['ball_tree', 'kd_tree', 'brute'],
    'knn_clf__estimator__n_neighbors': [5, 10, 15],
    'knn_clf__estimator__weights': ['uniform', 'distance'],
    #'knn_clf__metric': ['euclidian', 'manhattan']
}

cv = GridSearchCV(pipeline, param_grid=knn_est_params)


In [9]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('knn_clf', MultiOutputClassifier(KNeighborsClassifier()))
])
# sorted(pipeline.get_params().keys())

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
def display_results(cv, y_test, y_pred):
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)
    print("\nBest Parameters:", cv.best_params_)

In [ ]:
print('loading data')
X, y = load_data()
print('*'*80)

print('splitting data')
X_train, X_test, y_train, y_test = train_test_split(X, y)
print('*'*80)

print('fitting data')
cv.fit(X_train, y_train)
print('*'*80)

print('getting y-pred')
y_pred = cv.predict(X_test)
print('*'*80)

print('RESULTS')
display_results(cv, y_test, y_pred)
print('*'*80)

loading data
********************************************************************************
splitting data
********************************************************************************
fitting data


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
display_results(cv, y_test, y_pred)3

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.